# Winsorization


In [ ]:
# winsorize data function where upper and lower cutoff values can be different and winsorization can work on groups
# so if the index is a multi country or multi sector index those groups can be used and winsorization done within group
# this helps to avoid losing information about a particular section

def winsorize_var_m(variable=None,group=None,l_cutoff=None,u_cutoff=None,*args,**kwargs):
    varargin = winsorize_var_m.varargin
    nargin = winsorize_var_m.nargin

    nCos,nT=size(variable,nargout=2)
    w_variable=dot(NaN,ones(nCos,nT))
    for t in arange(1,nT).reshape(-1):
        u_group=concat([unique(group(arange(),t)),zeros(length(unique(group(arange(),t))),2)])
        index_stocks=concat([arange(1,nCos)]).T
        temp_var=concat([variable(arange(),t),zeros(nCos,1)])
        temp_group=group(isnan(temp_var(arange(),1)) == 0,t)
        index_stocks=index_stocks(isnan(temp_var(arange(),1)) == 0)
        temp_var=temp_var(isnan(temp_var(arange(),1)) == 0,arange())
        for i in arange(1,length(u_group(arange(),1))).reshape(-1):
            #getting the observations for group i
            temp_group_var=temp_var(temp_group == u_group(i,1))
            temp_index_stock=index_stocks(temp_group == u_group(i,1))
            #calculations
            if isempty(temp_group_var) == 0:
                breakp=prctile(temp_group_var,concat([l_cutoff,100 - u_cutoff]).T)
                inc_cutoff=1
                while abs(mean(temp_group_var(temp_group_var < logical_and(breakp(2,1),temp_group_var) > breakp(1,1)))) > dot(3,abs(median(temp_group_var(temp_group_var < logical_and(breakp(2,1),temp_group_var) > breakp(1,1))))):

                    breakp=prctile(temp_group_var,concat([l_cutoff + inc_cutoff,100 - (u_cutoff + inc_cutoff)]).T)
                    inc_cutoff=inc_cutoff + 1

                u_group[i,2]=mean(temp_group_var(temp_group_var < logical_and(breakp(2,1),temp_group_var) > breakp(1,1)))
                u_group[i,3]=std(temp_group_var(temp_group_var < logical_and(breakp(2,1),temp_group_var) > breakp(1,1)))
                temp_group_var[temp_group_var > (u_group(i,2) + dot(3,u_group(i,3)))]=(u_group(i,2) + dot(3,u_group(i,3)))
                temp_group_var[temp_group_var < (u_group(i,2) - dot(3,u_group(i,3)))]=(u_group(i,2) - dot(3,u_group(i,3)))
                w_variable[temp_index_stock,t]=temp_group_var
            else:
                u_group[i,2]=NaN
                u_group[i,3]=NaN  